The main solving big ridge regression problems

In [1]:
workspace()

In [2]:
using Plots

In [24]:
using StatsBase
using JLD  
include("../src/BigRidge.jl")

plot_outputs_Plots (generic function with 1 method)

In [25]:
#Basic parameters
## Basic parameters
maxiter=10^8;
max_time = 10;
max_epocs = 100;
printiters = true;
exacterror =true; 
tol = 10.0^(-5.0);
restol = 10.0^(-5.0);
skip_error_calculation = 25;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))

In [26]:
dataset = "a9a"
prob = defineProb(dataset);

../data/a9a.jld


In [27]:
norm(prob.A*prob.xsol - prob.b)

4.459713089533306e-11

In [28]:
options = MyOptions(0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror) 
OUTPUTS = [];

In [29]:
 s = options.sketchsize;
    rho = convert(Int64,floor(prob.n/s)); #hard coded density of rows
    ind = sample(1:prob.n,prob.n,replace=false); # shuffle all indices
    modns = mod(prob.n,s);
    divi = prob.n-modns;
    sold =s;
    if(modns!=0) 
        s= s+1;
    end
    indM = Array{Int64}(s,rho);# build a matrix indices with approx equally distributed number of indices over s rows
    indM[1:sold,:] =reshape(ind[1:divi], sold,rho);
    if(modns!=0)
        indM[s,1:modns] = ind[divi+1:end];
        indM[s,modns+1:end] =sample(1:prob.n, rho-modns,replace=false);
    end
    
    sigs = sample(1:2,prob.n,replace=true).*2.-3;

     SA = zeros(s,prob.n);
     Sb = zeros(s);

     for i =1:s
           SA[i,:] = sum(sigs[indM[i,:]].*prob.A[indM[i,:],:],1);
           Sb[i] = sum(sigs[indM[i,:]].*prob.b[indM[i,:]])
     end
     SAS = zeros(s,s); 
      for i =1:s
          SAS[i,:] = sum(sigs[indM[i,:]]'.*SA[:,indM[i,:]],2);
     end
     y = SAS\(SA*x-Sb);   # solving (S^TAS) y = (S^TAx-S^Tb)  
     for i =1:s #adding on S^T y
        x[indM[i,:]] = x[indM[i,:]]-sigs[indM[i,:]].*y[i];
     end

LoadError: InexactError()

In [30]:
# Testing Rademarcher Sketch
method_name = "rademacher";# ROS
options.sketchsize =5;
options.maxiter=200;
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

rademacher-5
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 7.84  |  4.28  | 0.4144 
 50  | 3.78  |  2.10  | 0.4210 
 75  | 2.16  |  1.38  | 0.4321 
100  | 1.56  |  1.20  | 0.4376 
125  | 1.24  |  0.82  | 0.4431 
150  | 0.96  |  0.64  | 0.4526 
175  | 0.77  |  0.53  | 0.4580 
200  | 0.69  |  0.40  | 0.4664 
  0.916280 seconds (577.59 k allocations: 88.303 MB, 2.06% gc time)


In [10]:
# Testing Hadamard Sketch
method_name = "Hadamard";# ROS
options.sketchsize =1; # Int(floor(log2(prob.n)));
options.maxiter=200;
options.skip_error_calculation =25;
@time  output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

flopsperiter: 247
Hadamard
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 27.02  |  16.06  | 1.1688 
 50  | 22.04  |  12.26  | 1.4007 
 75  | 15.05  |  7.83  | 1.5606 
100  | 12.02  |  8.17  | 2.0782 
125  | 8.88  |  6.00  | 2.2268 
150  | 6.86  |  5.88  | 2.3973 
175  | 5.65  |  4.03  | 2.5573 
200  | 4.90  |  3.24  | 2.7093 
  3.320107 seconds (8.62 M allocations: 1.007 GB, 6.08% gc time)


In [11]:
# Testing countmin 
options.max_time = 2; options.restol = 10.0^(-8); 
options.sketchsize =1;
options.skip_error_calculation =25;
method_name = "countmin";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

countmin
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 26.21  |  14.28  | 0.4756 
 50  | 15.99  |  13.35  | 0.5292 
 75  | 10.58  |  8.72  | 0.5706 
100  | 7.55  |  6.22  | 0.6037 
125  | 5.15  |  4.36  | 0.6231 
150  | 3.85  |  3.97  | 0.6415 
175  | 3.27  |  2.81  | 0.6607 
200  | 2.86  |  2.23  | 0.6799 
  0.703309 seconds (303.21 k allocations: 133.760 MB, 3.25% gc time)


In [12]:
# Testing CG 
options.max_time = 2; options.restol = 10.0^(-8); 
options.skip_error_calculation =25;
method_name = "CG";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CG
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 0.03  |  0.07  | 0.0859 
 50  | 0.00  |  0.06  | 0.1135 
 75  | 0.00  |  0.01  | 0.1139 
  0.132333 seconds (59.56 k allocations: 3.229 MB)


In [13]:
# Testing CD 
options.max_time = 10; options.restol = 10.0^(-8); options.maxiter=200;
method_name = "CD";# gradient descent
options.sketchsize = 5#floor((prob.n)^(1/2));
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CD-0
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 7.22  |  5.35  | 0.1365 
 50  | 2.36  |  1.73  | 0.1368 
 75  | 1.21  |  1.23  | 0.1371 
100  | 0.77  |  0.85  | 0.1374 
125  | 0.50  |  0.77  | 0.1378 
150  | 0.29  |  0.63  | 0.1381 
175  | 0.23  |  0.48  | 0.1384 
200  | 0.17  |  0.23  | 0.1387 
  0.162747 seconds (60.65 k allocations: 3.829 MB)


In [14]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename)-OUTPUTS.jld", "OUTPUTS", OUTPUTS)

In [15]:
#using GR
using Plots
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

a9a


In [16]:
# Loading saved problem data
probname = "w8a"#"gisette_scale-ridge";  
test = load("../data/$(probname)-OUTPUTS.jld")
OUTPUTS = test["OUTPUTS"];